In [4]:
import cv2
import numpy as np 
import os 
from random import shuffle 
import pandas as pd 
import tqdm 
import tensorflow as tf 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Activation, Dropout, Dense
train_dir = '/Users/mousumikapoor/Desktop/train'
test_dir = '/Users/mousumikapoor/Desktop/test1'
img_width = 150
img_height = 150
train_images_dogs_cats = os.listdir(train_dir)[0:1300] + os.listdir(train_dir)[12500:13800] 
def label_image(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': 
        return 0
    elif word_label == 'dog': 
        return 1

def create_train_data(): 
    train_data_image = []
    train_data_label = []
    for img in train_images_dogs_cats:
        label = label_image(img)
        path = os.path.join(train_dir, img)
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        resized_image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
        train_data_image.append(resized_image)
        train_data_label.append(np.array(label))
    return train_data_image, train_data_label 

def process_test_data():
    testing_data = []
    for img in os.listdir(test_dir):
        path = os.path.join(test_dir,img)
        img_num = img.split('.')[0]
        image = cv2.imread(path,cv2.IMREAD_COLOR)
        resized_image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
        testing_data.append(resized_image)
    return testing_data

X,Y = create_train_data()
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3)))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(64, (3, 3)))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(64))
classifier.add(Activation('relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(1))
classifier.add(Activation('sigmoid'))
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)
history = classifier.fit_generator(
    train_generator, 
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = 30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)
classifier.save_weights('model_weights.h5')
classifier.save('model_keras.h5')


#X_test = process_test_data()
#Y_test = []
#nb_test_samples = len(X_test)
#test_datagen = ImageDataGenerator(rescale=1. / 255)
#test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
#prediction_probabilities = classifier.predict_generator(test_generator, steps = nb_test_samples // batch_size, verbose=1)
#counter = range(1, len(test_images_dogs_cats) + 1)
#solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
#cols = ['label']
#for col in cols:
    #solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

#solution.to_csv("dogsVScats.csv", index = False)

Epoch 1/30
130/130 [==============================] - 46s 353ms/step - loss: 0.7056 - acc: 0.5207 - val_loss: 0.6773 - val_acc: 0.5156
Epoch 2/30
130/130 [==============================] - 41s 312ms/step - loss: 0.6776 - acc: 0.5774 - val_loss: 0.6530 - val_acc: 0.5655
Epoch 3/30
130/130 [==============================] - 46s 356ms/step - loss: 0.6473 - acc: 0.6322 - val_loss: 0.5799 - val_acc: 0.6865
Epoch 4/30
130/130 [==============================] - 45s 343ms/step - loss: 0.6212 - acc: 0.6688 - val_loss: 0.6260 - val_acc: 0.6091
Epoch 5/30
130/130 [==============================] - 42s 321ms/step - loss: 0.6070 - acc: 0.6798 - val_loss: 0.5868 - val_acc: 0.7083
Epoch 6/30
130/130 [==============================] - 41s 314ms/step - loss: 0.5853 - acc: 0.7048 - val_loss: 0.5486 - val_acc: 0.6925
Epoch 7/30
130/130 [==============================] - 41s 316ms/step - loss: 0.5739 - acc: 0.7029 - val_loss: 0.5248 - val_acc: 0.7262
Epoch 8/30
130/130 [==============================] - 4

In [5]:
from keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os 
test_dir = '/Users/mousumikapoor/Desktop/test1'
test_images_dogs_cats = [test_dir+i for i in os.listdir(test_dir)]
classifier = load_model('model_keras.h5')
X_test = process_test_data()
Y_test = []
nb_test_samples = len(X_test)
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = classifier.predict_generator(test_generator, steps = nb_test_samples // batch_size, verbose=1)
for index, probability in enumerate(prediction_probabilities):
    image_path = test_images_dogs_cats[index]
    img = mpimg.imread(image_path)
    with open(test_dir, "a") as fh:
        fh.write(str(probability[0]) + " for: " + image_path + "\n")
    plt.imshow(img)
    if probability > 0.5:
        plt.title("%.2f" % (probability[0]*100) + "% dog")
    else:
        plt.title("%.2f" % ((1-probability[0])*100) + "% cat")
    plt.show()


#counter = range(1, 12497)
#solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
#cols = ['label']
#for col in cols:
    #solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)
#solution.to_csv("dogsVScats.csv", index = False)

781/781 [==============================] - 337s 431ms/step


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mousumikapoor/Desktop/test19733.jpg'

In [12]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
img_width, img_height = 150, 150
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

train_data_dir = '/Users/mousumikapoor/Desktop/data/train_data'
validation_data_dir = '/Users/mousumikapoor/Desktop/data/validation_data'

def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet', input_shape = (150,150,3))
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, steps = nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights('bottleneck_fc_model.h5')
save_bottleneck_features()
train_top_model()

Found 0 images belonging to 0 classes.


InvalidArgumentError: input and filter must have the same depth: 3 vs 64
	 [[{{node block1_conv2_7/Relu}}]]

In [18]:
batch_size = 32
from keras.applications import InceptionResNetV2
conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))
from keras import layers
from keras import models
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  
conv_base.trainable = False
from keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

train_datagen = ImageDataGenerator(rescale=1./255,  
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(np.array(X_train), Y_train,batch_size=batch_size)
val_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)
history = model.fit_generator(train_generator,
                              steps_per_epoch= nb_train_samples // batch_size,
                              epochs=20,
                              validation_data=val_generator,
                              validation_steps= nb_validation_samples // batch_size)

Epoch 1/20
 6/65 [=>............................] - ETA: 22:02 - loss: 0.7615 - acc: 0.4896

KeyboardInterrupt: 